In [38]:
from fastai.vision.all import *

# Dataset preparations

In [39]:
path = untar_data(URLs.MNIST)
path.ls()

(#2) [Path('/storage/data/mnist_png/training'),Path('/storage/data/mnist_png/testing')]

In [40]:
(path/'training').ls()

(#10) [Path('/storage/data/mnist_png/training/0'),Path('/storage/data/mnist_png/training/2'),Path('/storage/data/mnist_png/training/9'),Path('/storage/data/mnist_png/training/8'),Path('/storage/data/mnist_png/training/7'),Path('/storage/data/mnist_png/training/1'),Path('/storage/data/mnist_png/training/5'),Path('/storage/data/mnist_png/training/4'),Path('/storage/data/mnist_png/training/6'),Path('/storage/data/mnist_png/training/3')]

In [41]:
img = Image.open((path/'training/0').ls()[0])
tensor(img).shape

torch.Size([28, 28])

In [43]:
training_x = []
training_y = []

for sub_folder in (path/'training').ls():
    category = sub_folder.name
    print(category)
    for img in sub_folder.glob('*.png'):
        training_x.append(tensor(Image.open(img)))
        training_y.append(int(category))

0
2
9
8
7
1
5
4
6
3


In [57]:
training_tensor_x = torch.stack(training_x).view(-1, 28*28).float()/255.
training_tensor_x.shape

torch.Size([60000, 784])

In [58]:
training_tensor_y = tensor(training_y)
training_tensor_y.shape

torch.Size([60000])

In [59]:
save_path = Path('/storage/data/mnist_tensor')
save_path.ls()

(#1) [Path('/storage/data/mnist_tensor/training_x.save')]

In [60]:
torch.save(training_tensor_x, save_path/'training_x.save')
torch.save(training_tensor_y, save_path/'training_y.save')

In [71]:
def setup_mnist():
    save_path = Path('/storage/data/mnist_tensor') 
    
    target_files = ['training_x.save', 'training_y.save', 'testing_x.save', 'testing_y.save']
    loaded = []

    all_valid = np.all([(save_path/target_file).is_file() for target_file in target_files])
    if not all_valid:
        print('Setup from scratch')
        path = untar_data(URLs.MNIST)
        
        for mode in ['training', 'testing']:
            print(mode)
            x = []
            y = []

            for sub_folder in (path/mode).ls():
                category = sub_folder.name
                print(category)
                for img in sub_folder.glob('*.png'):
                    x.append(tensor(Image.open(img)))
                    y.append(int(category))
                    
            x = torch.stack(x).view(-1, 28*28).float()/255.
            y = tensor(y)
            
            torch.save(x, save_path/(mode+'_x.save'))
            torch.save(y, save_path/(mode+'_y.save'))
        
    
    return [torch.load(save_path/target_file) for target_file in target_files]    

In [73]:
train_x, train_y, test_x, test_y = setup_mnist()